In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from sklearn.model_selection import train_test_split
import os
import joblib
import numpy as np
import pandas as pd

In [2]:
ws = Workspace.from_config()
# choose a name for experiment
experiment_name = 'labor-prediction-experiment'
experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RXRUZXU9T to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset, Datastore
from azureml.data.datapath import DataPath

datastore_name='workspaceblobstore'
datastore=Datastore.get(ws,datastore_name)

datastore_path = [(datastore, 'UI/01-19-2021_090500_UTC/LaborPredictionData6.csv')]
ds = Dataset.Tabular.from_delimited_files(path=datastore_path)

ds = ds.take(3000).to_pandas_dataframe()

In [4]:
from train import clean_data
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=.8)

#y_train=y_train.reshape(1728, 1)
#train = np.concatenate((x_train,y_train), axis=1)
train=pd.concat([x_train, y_train], axis=1)

In [14]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "max_concurrent_iterations":3,
    "primary_metric": 'r2_score'
}

automl_config = AutoMLConfig(
    task='regression',
    training_data=train,
    label_column_name='TotalInstallationTime',
    featurization='auto',
    n_cross_validations=10,
    debug_log = "automl_errors.log",
    **automl_settings
    )

In [15]:
#TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_2aaa0b2f-6984-458d-a646-e2b25a95506c

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

***************************************************

In [16]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [17]:
best_mlrun, fitted_mlmodel = remote_run.get_output()
print(best_mlrun)
print(fitted_mlmodel)
joblib.dump(fitted_mlmodel, "automlmodel.pkl")

Run(Experiment: labor-prediction-experiment,
Id: AutoML_2aaa0b2f-6984-458d-a646-e2b25a95506c_33,
Type: None,
Status: Completed)
RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             lo

['automlmodel.pkl']

In [18]:
#register the model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

description = 'Best AutoML Model'
#model = best_mlrun.register_model(description=description, workspace = , model_name='automl_pred', model_path='./models')
model = Model.register(model_path="automlmodel.pkl", model_name = "automl_pred", description=description, workspace=ws)
print(model.name, model.id, model.version, sep='\t')



Registering model automl_pred
automl_pred	automl_pred:2	2


In [19]:
#define inference config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

#create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

#Define the packages needed by the model and scripts
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("py-xgboost")
#You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("keras")
conda_dep.add_pip_package("gensim")
conda_dep.add_pip_package("azureml-automl-core")
conda_dep.add_pip_package("azureml-automl-runtime")
conda_dep.add_pip_package("packaging")


myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

In [21]:
#deploy as webservice
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.train.automl import *

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)
#deployment_config = LocalWebservice.deploy_configuration()

model = Model(ws, name='automl_pred')
print(model)
service = Model.deploy(ws, 'laborpredservice', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.get_logs)
print(service.state)
print("scoring URI: " + service.scoring_uri)


Model(workspace=Workspace.create(name='quick-starts-ws-135316', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-135316'), name=automl_pred, id=automl_pred:2, version=2, tags={}, properties={})
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.........

KeyboardInterrupt: 

In [ ]:
#send request

import requests
import json
from pandas import DataFrame

scoring_uri = service.scoring_uri
key = 'xlEe0aHu7gegnAEgyATae6cGEAEfhMp6'
headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key}
#test_data = json.dumps({'data': [{'heightwholenum':67, 'framemat':'ALUMINUM', 'framefunc':'DOOR', 'widthWholeNum':24, 'lift':0, 'itemset':'SNAP BEADS', 'locationtype':9, 'leadmechanic':118}]})
#data = json.loads([67, 'ALUMINUM', 'DOOR', 24, 0, 'SNAP BEADS', 9, 118])['data']
#['heightwholenum','widthWholeNum','lift','LongCarry', 
#'itemset_','itemset_OTHER', 'itemset_PRESSURE PLATE','itemset_PUTTY','itemset_SCREW BEADS','itemset_SNAP BEADS','itemset_VINYL BEADS', 'itemset_WOOD BEADS','itemset_WRAP AROUND FRAME','itemset_WRAP AROUND RUBBER',
#'itemset_flush glaze', 'leadmechanic_6','leadmechanic_11','leadmechanic_62','leadmechanic_90','leadmechanic_118','leadmechanic_211', 'leadmechanic_228','leadmechanic_OTHER',
#'locationtype_0','locationtype_1','locationtype_2','locationtype_4','locationtype_5','locationtype_6','locationtype_7','locationtype_9','locationtype_10','locationtype_11','locationtype_12', 'locationtype_13',
#'framemat_','framemat_ALUMINUM','framemat_ANDERSEN','framemat_OTHER','framemat_SASH RAIL','framemat_VINYL','framemat_WOOD','framefunc_BOTTOM DH','framefunc_BOTTOM SH', 'framefunc_CASEMENT', 
#'framefunc_CURTAIN WALL', 'framefunc_FIXED','framefunc_FLUSH GLAZE', 'framefunc_OTHER', 'framefunc_SASH', 'framefunc_SIDELITE', 'framefunc_SLIDER','framefunc_TOP DH', 'framefunc_TOP SH','framefunc_TRANSOM']]
    
input_string = {'data':[{"heightwholenum":67,"widthWholeNum":24,"lift":0,"LongCarry":0,"itemset_":0,"itemset_OTHER":0,"itemset_PRESSURE PLATE":0,"itemset_PUTTY":0,"itemset_SCREW BEADS":0,"itemset_SNAP BEADS":1,
"itemset_VINYL BEADS":0,"itemset_WOOD BEADS":0,"itemset_WRAP AROUND FRAME":0,"itemset_WRAP AROUND RUBBER":0,"itemset_flush glaze":0,"leadmechanic_6":0,"leadmechanic_11":0,"leadmechanic_62":0,"leadmechanic_90":0,
"leadmechanic_118":1,"leadmechanic_211":0,"leadmechanic_228":0,"leadmechanic_OTHER":0,"locationtype_0":0,"locationtype_1":0,"locationtype_2":0,"locationtype_4":0,"locationtype_5":0,"locationtype_6":0,
"locationtype_7":0,"locationtype_9":1,"locationtype_10":0,"locationtype_11":0,"locationtype_12":0,"locationtype_13":0,"framemat_":0,"framemat_ALUMINUM":1,"framemat_ANDERSEN":0,"framemat_OTHER":0,
"framemat_SASH RAIL":0,"framemat_VINYL":0,"framemat_WOOD":0,"framefunc_BOTTOM DH":0,"framefunc_BOTTOM SH":0,"framefunc_CASEMENT":0,"framefunc_CURTAIN WALL":0,"framefunc_FIXED":0,"framefunc_FLUSH GLAZE":0,
"framefunc_OTHER":0,"framefunc_SASH":0,"framefunc_SIDELITE":0,"framefunc_SLIDER":0,"framefunc_TOP DH":0,"framefunc_TOP SH":0,"framefunc_TRANSOM":0}]}
data = json.dumps(input_string)

#data = "{\"data\": [" + str(list(input_string)) + "]}"
#data = json.dumps(np.array(input_string).tolist())
#data = pd.DataFrame.from_dict(data)

response = requests.post(scoring_uri, data=data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.text)